In [0]:
%run ../config

In [0]:
# Importações

from pyspark.sql import SparkSession
import requests
import json


In [0]:
# Inicializar Spark
spark = SparkSession.builder.getOrCreate()

In [0]:
# Coleta os dados paginados da API
dados = []

def fetch_page(page: int):
    url = (
        "https://cadastro.museus.gov.br/wp-json/tainacan/v2/collection/208/items/"
        f"?perpage=96&order=DESC&orderby=meta_value&metakey=222&paged={page}"
    )
    r = requests.get(url, timeout=30)
    r.raise_for_status()
    return r.json().get("items", [])

# Coletar 42 páginas
for p in range(1, 43):
    page_items = fetch_page(p)
    dados.extend(page_items)
    print(f"Página {p:02d}: {len(page_items)} itens")

In [0]:
display(df_bronze)

In [0]:
# Transformação em RDD JSON
rdd_json = spark.sparkContext.parallelize([json.dumps(item) for item in dados])

# Leitura como DataFrame Spark
df_bronze = spark.read.json(rdd_json)

In [0]:
# Schema e amostra
df_bronze.printSchema()
df_bronze.show(3, truncate=False)

In [0]:
# Salvar dados na camada Bronze
df_bronze.write.mode("overwrite").parquet("/mnt/eoliveira/bronze/museusbr")

print("✔ Bronze salvo com sucesso!")


In [0]:
df_bronze.select("metadata.location.value_as_string").show(10, truncate=False)
